In [3]:
import requests
import pandas as pd
from prettytable import PrettyTable

API_URL = "https://leetcode.com/graphql"

QUERY = """
query getUserProfile($username: String!) {
    matchedUser(username: $username) {
        username
        profile {
            ranking
        }
        submitStats {
            acSubmissionNum {
                difficulty
                count
            }
        }
        badges {
            displayName
        }
    }
}
"""

def fetch_user_data(username):
    """Fetch data for a single LeetCode user."""
    try:
        response = requests.post(API_URL, json={"query": QUERY, "variables": {"username": username}})
        data = response.json()
        
        # Check if the user exists
        user = data["data"]["matchedUser"]
        if not user:
            return {
                "Username": username,
                "Ranking": float('inf'),  # To sort "N/A" ranks at the end
                "Easy": "N/A",
                "Medium": "N/A",
                "Hard": "N/A",
                "Total Solved": "N/A",
                "Badges": "N/A"
            }
        
        # Extract details
        username = user["username"]
        ranking = user["profile"]["ranking"] or float('inf')  # Convert "None" to inf for sorting
        
        # Extract problem counts by difficulty
        problems_by_difficulty = {entry["difficulty"].lower(): entry["count"] for entry in user["submitStats"]["acSubmissionNum"]}
        easy = problems_by_difficulty.get("easy", 0)
        medium = problems_by_difficulty.get("medium", 0)
        hard = problems_by_difficulty.get("hard", 0)
        total_solved = easy + medium + hard
        
        # Collecting badges and separating them by commas
        badges = ", ".join([badge["displayName"] for badge in user["badges"]]) or "None"
        
        return {
            "Username": username,
            "Ranking": ranking,
            "Easy": easy,
            "Medium": medium,
            "Hard": hard,
            "Total Solved": total_solved,
            "Badges": badges,
        }
    except Exception as e:
        return {
            "Username": username,
            "Ranking": float('inf'),
            "Easy": "Error",
            "Medium": "Error",
            "Hard": "Error",
            "Total Solved": "Error",
            "Badges": str(e)
        }

def fetch_bulk_data(usernames):
    """Fetch data for multiple LeetCode users."""
    results = []
    for username in usernames:
        results.append(fetch_user_data(username))
    # Sort results by ranking (ascending)
    results.sort(key=lambda x: x["Ranking"])
    return results

def save_to_csv(data, filename="leetcode_data.csv"):
    """Save fetched data to a CSV file."""
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding="utf-8")
    print(f"Data saved to {filename}")

def generate_html(data, output_file="leetcode_data.html"):
    """Generate an HTML page to display the data with a styled table."""
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>LeetCode User Data</title>
        <style>
            body {
                background-color: #F5F0CD;  /* Light yellow background */
                color: black;
                font-family: Arial, sans-serif;
            }
            table {
                width: 80%;
                margin: 20px auto;
                border-collapse: collapse;
                box-shadow: 0 2px 15px rgba(64,64,64,.7);
            }
            th, td {
                padding: 12px;
                border: 1px solid #ddd;
                text-align: center;
            }
            th {
                background-color: #B1F0F7;  /* Light blue header */
                color: black;
            }
            tr:nth-child(even) {
                background-color: #81BFDA;  /* Light turquoise rows */
            }
            tr:nth-child(odd) {
                background-color: #F5F0CD;  /* Light yellow alternate rows */
            }
        </style>
    </head>
    <body>
        <h1 style="text-align: center;">LeetCode User Data</h1>
        <table>
            <thead>
                <tr>
                    <th>Username</th>
                    <th>Ranking</th>
                    <th>Easy</th>
                    <th>Medium</th>
                    <th>Hard</th>
                    <th>Total Solved</th>
                    <th>Badges</th>
                </tr>
            </thead>
            <tbody>
    """
    for record in data:
        html_content += f"""
            <tr>
                <td>{record["Username"]}</td>
                <td>{'N/A' if record['Ranking'] == float('inf') else record['Ranking']}</td>
                <td>{record["Easy"]}</td>
                <td>{record["Medium"]}</td>
                <td>{record["Hard"]}</td>
                <td>{record["Total Solved"]}</td>
                <td>{record["Badges"]}</td>
            </tr>
        """

    html_content += """
            </tbody>
        </table>
    </body>
    </html>
    """

    with open(output_file, "w", encoding="utf-8") as file:
        file.write(html_content)
    print(f"HTML file generated: {output_file}")

if __name__ == "__main__":
    # Read usernames from an input Excel file
    input_file = input("Enter the path to the input Excel file (e.g., input.xlsx): ").strip()
    try:
        # Read the Excel file
        df_input = pd.read_excel(input_file)
        
        # Ensure the Excel file has a column named "Username"
        if "Username" not in df_input.columns:
            print("The Excel file must have a column named 'Username'. Please fix the file and try again.")
            exit()
        
        # Extract usernames from the Excel file
        usernames = df_input["Username"].dropna().tolist()
        print(f"Found {len(usernames)} usernames in the Excel file.")

        # Fetch data
        print("Fetching data...")
        data = fetch_bulk_data(usernames)

        # Display data using PrettyTable
        table = PrettyTable()
        table.field_names = ["Username", "Ranking", "Easy", "Medium", "Hard", "Total Solved", "Badges"]

        for record in data:
            table.add_row([
                record["Username"],
                'N/A' if record['Ranking'] == float('inf') else record['Ranking'],
                record["Easy"],
                record["Medium"],
                record["Hard"],
                record["Total Solved"],
                record["Badges"]
            ])
        table.align = "l"

        print("\nFetched Data:")
        print(table)

        # Save data to a CSV file
        save_to_csv(data)

        # Generate HTML file
        generate_html(data)

    except FileNotFoundError:
        print("Input file not found. Please check the file path and try again.")
    except Exception as e:
        print(f"An error occurred: {e}")


Enter the path to the input Excel file (e.g., input.xlsx):  C:\Users\venug\Downloads\user names.xlsx


Found 57 usernames in the Excel file.
Fetching data...

Fetched Data:
+----------------------+---------+------+--------+------+--------------+-------------------------------------------------------------------+
| Username             | Ranking | Easy | Medium | Hard | Total Solved | Badges                                                            |
+----------------------+---------+------+--------+------+--------------+-------------------------------------------------------------------+
| Aravind_kammari      | 175947  | 242  | 167    | 9    | 418          | 200 Days Badge 2024, 100 Days Badge 2024, 50 Days Badge 2024      |
| dheerajsiripragada   | 184250  | 237  | 166    | 6    | 409          | 200 Days Badge 2024, 100 Days Badge 2024, 50 Days Badge 2024      |
| nirmal7463           | 195402  | 251  | 129    | 15   | 395          | 100 Days Badge 2024, 50 Days Badge 2024                           |
| sans_sand            | 222192  | 180  | 164    | 23   | 367          | 100 Days Ba